In [2]:
import sqlite3
import psycopg2
import pandas as pd
import numpy as np
import math
from DataBase_functions import *
from DataBase_functions_testing import *

#Google sheed API
import gspread
from google.oauth2.service_account import Credentials
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload

<p style="font-size: 30px; text-align: center; color: #20a7e5;">Creating data-frame</p>

<p style="font-size: 25px; color: #208ee5">Registro actividad todos</p>

In [3]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "12qLRAp7C9qzcoWkyUTJAVMhRHMFfwYec094J8CU2t7c" #Registro actividad todos: 12qLRAp7C9qzcoWkyUTJAVMhRHMFfwYec094J8CU2t7c

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #Dataframe con los datos de todos los meses
    df = pd.DataFrame(columns=["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]) 
    meses = ["Enero", "Febrero", "Marzo", "Abril", "Mayo", "Junio", "Julio", "Agosto", "Septiembre", "Octubre", "Noviembre", "Diciembre"]
    for sheet in sheets:
        sheet_name = sheet.title 
        if sheet_name in meses:
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]))
            df = pd.concat([df, df_aux], ignore_index=True)
        else: 
            print(sheet_name, "is not a month")
            pass
    #df = df.drop_duplicates(subset='Código', keep='first')
    print(df.shape, "\n", df.columns)

    #Dataframe with 2024 sheet data
    sheet_2024 = spreadsheet.worksheet("2024")
    df_2024 = pd.DataFrame(sheet_2024.get_all_records(expected_headers=["Fecha de alta", "Fecha incurrida", "Fecha de fin", "UUAA", "Código", "SDATOOL", "Feature", "Petición", "Geografía", "Gestor BBDD", "Ámbito", "Responsable", "Validada", "Horas", "Comentarios"]))
    #df_2024 = df_2024.drop_duplicates(subset='Código', keep='first')
    print(df_2024.shape, "\n", df_2024.columns)

    #Concatenate both dataframes
    df = pd.concat([df, df_2024], ignore_index=True)
    df = df.drop_duplicates(subset="Código", keep="first")
    print(df.shape)

    #Dataframe validation
    df.rename(columns={"Validada": "validada", "Responsable": "responsable", "Gestor DDBB": "DDBB", "Geografía": "geography", "Petición": "petition_code", "Feature": "feature", "SDATOOL": "sdatool", "Código": "DQDP_code", "Gestor BBDD": "DDBB", "Horas": "duration_time", "Comentarios": "description", "Fecha de alta": "fecha_in", "Fecha de fin": "fecha_out"}, inplace=True)

    df["fecha_in"] = df["fecha_in"].astype(str)
    df["Fecha incurrida"] = df["Fecha incurrida"].astype(str)
    df["fecha_out"] = df["fecha_out"].astype(str)
    df["UUAA"] = df["UUAA"].astype(str)
    df["DQDP_code"] = df["DQDP_code"].astype(str)
    df["sdatool"] = df["sdatool"].astype(str)
    df["feature"] = df["feature"].astype(str)
    df["geography"] = df["geography"].astype(str)
    df["DDBB"] = df["DDBB"].astype(str)
    df["Ámbito"] = df["Ámbito"].astype(str)
    df["responsable"] = df["responsable"].astype(str)
    df["validada"] = df["validada"].astype(str)
    df["duration_time"] = pd.to_numeric(df["duration_time"], errors="coerce")
    df["description"] = df["description"].astype(str)
    df["petition_code"] = df["petition_code"].astype(str)

    df["version_date"] = "Nan"
    df["version"] = "Nan"
    df["petition_arq"] = "Nan"
    df["dev_master"] = "Nan"
    df["estado"] = "Nan"

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['2024 totales - Peticiones', '2024 totales - Peticiones - Pro', '2024', 'Gráficos Semanales - Procesos', '2024 - Procesos', 'Abril', 'Mayo', 'Julio', 'Junio', 'Agosto', 'Septiembre', 'Octubre', 'Noviembre', 'Diciembre', 'Responsables']
2024 totales - Peticiones is not a month
2024 totales - Peticiones - Pro is not a month
2024 is not a month
Gráficos Semanales - Procesos is not a month
2024 - Procesos is not a month
Responsables is not a month
(4371, 18) 
 Index(['Fecha de alta', 'Fecha incurrida', 'Fecha de fin', 'UUAA', 'Código',
       'SDATOOL', 'Feature', 'Petición', 'Geografía', 'Gestor BBDD', 'Ámbito',
       'Responsable', 'Validada', 'Horas', 'Comentarios', 'Estado', 'Días',
       ''],
      dtype='object')
(1974, 18) 
 Index(['Fecha de alta', 'Fecha incurrida', 'Fecha de fin', 'UUAA', 'Código',
       'SDATOOL', 'Feature', 'Petición', 'Geografía', 'Gestor BBDD', 'Ámbito',
       'Responsable', 'Validada', 'Horas', 'Comentarios', 'Estado', 'Días',
       ''

<p style="font-size: 25px; color: #208ee5; text-align: center;">Peticiones globales Oracle, Elastic Search y Mongo DB</p>

<p style="font-size: 25px; color: #208cc5;;">Oracle Physics</p>

In [4]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1PZKCpybEechpbwP-H3khhUihHqwUqgbi_KkM6iCxhuE" #Peticiones globales Oracle: "1PZKCpybEechpbwP-H3khhUihHqwUqgbi_KkM6iCxhuE"

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #DataFrame con los datos del excel
    df_OG = pd.DataFrame(columns=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"])

    for sheet in sheets: 
        try: 
            sheet_name = sheet.title
            
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"]))
            df_OG = pd.concat([df_OG, df_aux], ignore_index=True)
        except gspread.exceptions.APIError as e:
            if e.response.status_code == 429: 
                wait_time = 6
                print(f"Quota exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else: 
                pass
    
    #Creating DDBB and geography column
    df_OG["DDBB"] = "Oracle Physics"
    df_OG["geography"] = "Global"

    print(df_OG.shape)

    df_OG["UUAA"] = df_OG["UUAA"].astype(str)
    df_OG["Petición Jira"] = df_OG["Petición Jira"].astype(str)
    df_OG["Versión Repo"] = df_OG["Versión Repo"].astype(str)
    df_OG["Petición ARQ"] = df_OG["Petición ARQ"].astype(str)
    df_OG["Fecha cierre"] = df_OG["Fecha cierre"].astype(str)
    df_OG["Comentarios"] = df_OG["Comentarios"].astype(str)
    df_OG["Path"] = df_OG["Path"].astype(str)
    df_OG["DDBB"] = df_OG["DDBB"].astype(str)
    df_OG["geography"] = df_OG["geography"].astype(str)

    df_OG.rename(columns={"Petición Jira": "petition_code", "Versión Repo": "version", "Petición ARQ": "petition_arq", "Fecha cierre": "fecha_out", "Comentarios": "description"}, inplace=True)

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['APIC', 'ATAU', 'B1DS', 'B43T', 'BJP9', 'CCUS', 'DAMS', 'DMTO', 'EGRP', 'EI4M', 'ESCD', 'FZHI', 'GDEL', 'GHDC', 'GJSX', 'GSIN', 'GUAR', 'H9IY', 'HFPP', 'HLCY', 'HPGR', 'IMPT', 'J6G7', 'K5VI', 'KAPI', 'KARC', 'KBGE', 'KBTQ', 'KCCX', 'KCDO', 'KCMC', 'KCMS', 'KCNC', 'KCNS', 'KCOG', 'KCOL', 'KCSN', 'KCSP', 'KDCL', 'KDFH', 'KDMG', 'KERF', 'KESJ', 'KFGC', 'KFRE', 'KFUL', 'KGCC', 'KGCI', 'KGTP', 'KIFV', 'KIVS', 'KLIM', 'KLNE', 'KMET', 'KMIC', 'KMOL', 'KMRK', 'KMYC', 'KNWD', 'KOMN', 'KONB', 'KPAG', 'KPAD', 'KPAY', 'KPDA', 'KPDR', 'KPFE', 'KPFM', 'KPIG', 'KPPS', 'KPST', 'KRBA', 'KRCE', 'KSAG', 'KSAN', 'KSDD', 'KSPH', 'KSRO', 'KUSU', 'KVCS', 'KXSG', 'KYSZ', 'KYOP', 'KYUB', 'MCRR', 'MFCG ', 'MK0W', 'MDDM', 'NI0C', 'O43T', 'PGVC', 'PIVU', 'PMSC', 'PMSV', 'P6ES', 'QWJR', 'QWPJ', 'RLR7', 'SWP3', 'S47B', 'UGX2', 'V34F', 'VAMO', 'W1BD', 'XBOB', 'ZGR7', 'XMBR', 'XR6Z', 'ZDNM', 'ZNXD']
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Re

<p style="font-size: 25px; color: #208cc5;;">Elastic Search</p>

In [5]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1PXFKrrJVs12XpmgV5DmnBvjxENQ3hobwLs-bdDf8OIE" #Peticiones globales Mongo: "1PXFKrrJVs12XpmgV5DmnBvjxENQ3hobwLs-bdDf8OIE"

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #DataFrame con los datos del excel
    df_EG = pd.DataFrame(columns=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"])

    for sheet in sheets: 
        try: 
            sheet_name = sheet.title
            
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"]))
            df_EG = pd.concat([df_EG, df_aux], ignore_index=True)
        except gspread.exceptions.APIError as e:
            if e.response.status_code == 429: 
                wait_time = 6
                print(f"Quota exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else: 
                pass
    
    #Creating DDBB and geography column
    df_EG["DDBB"] = "Oracle Physics"
    df_EG["geography"] = "Global"

    #df = df_OG.drop_duplicates(subset='Código', keep='first')
    print(df_EG.shape)

    df_EG["UUAA"] = df_EG["UUAA"].astype(str)
    df_EG["Petición Jira"] = df_EG["Petición Jira"].astype(str)
    df_EG["Versión Repo"] = df_EG["Versión Repo"].astype(str)
    df_EG["Petición ARQ"] = df_EG["Petición ARQ"].astype(str)
    df_EG["Fecha cierre"] = df_EG["Fecha cierre"].astype(str)
    df_EG["Comentarios"] = df_EG["Comentarios"].astype(str)
    df_EG["Path"] = df_EG["Path"].astype(str)
    df_EG["DDBB"] = df_EG["DDBB"].astype(str)
    df_EG["geography"] = df_EG["geography"].astype(str)

    df_EG.rename(columns={"Petición Jira": "petition_code", "Versión Repo": "version", "Petición ARQ": "petition_arq", "Fecha cierre": "fecha_out", "Comentarios": "description"}, inplace=True)

except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['ATAU', 'F30V', 'EI4M', 'GHDC', 'GUAR', 'H9IY', 'IMPT', 'J6G7', 'JBSY', 'KAGE', 'KAPI', 'KBTQ', 'KCNS', 'KHR1', 'KIFV', 'KLNE', 'KMOL', 'KREC', 'KTIN', 'KTRA', 'KTR2', 'KXSG', 'KYGU', 'KYOP', 'L1WI', 'NI0C', 'O43T', 'OPEI', 'PGVC', 'S47B', 'XBMR', 'XPI2', 'YM3X', 'ZNXD']
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
Quota exceeded. Retrying in 6 seconds...
(230, 10)


<p style="font-size: 25px; color: #208cc5;">Mongo DB</p>

In [6]:
# Email service account that need to share the google sheet
service_account_email = "matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com"
#email that need to share the google sheet = matiasblaquier@theta-voyager-406314.iam.gserviceaccount.com

scope = [
    'https://spreadsheets.google.com/feeds', 
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
    ]

credentials = "credentials.json"

try: 
    #Cargar credenciales
    creds = ServiceAccountCredentials.from_json_keyfile_name(credentials, scope)

    #Autorizar gspread con las credenciales
    client = gspread.authorize(creds)

    # ID de la hoja de cálculo 
    srpeadsheet_id = "1-mO_2AFa0J4VBlONUFPuQ7IfTRNxBb3BbPyEg_D-FPs" #Peticiones globales Mongo: "1-mO_2AFa0J4VBlONUFPuQ7IfTRNxBb3BbPyEg_D-FPs"

    # Open the Google Sheet file by ID
    spreadsheet = client.open_by_key(srpeadsheet_id)

    # List all sheets file
    sheets = spreadsheet.worksheets()
    print(f"Available sheeds: {[sheet.title for sheet in sheets]}")

    #DataFrame con los datos del excel
    df_MG = pd.DataFrame(columns=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"])

    for sheet in sheets: 
        try: 
            sheet_name = sheet.title
            
            df_aux = pd.DataFrame(sheet.get_all_records(expected_headers=["UUAA", "Petición Jira", "Versión Repo", "Petición ARQ", "Fecha cierre", "Comentarios", "Path"]))
            df_MG = pd.concat([df_MG, df_aux], ignore_index=True)
        except gspread.exceptions.APIError as e:
            if e.response.status_code == 429: 
                wait_time = 6
                print(f"Quota exceeded. Retrying in {wait_time} seconds...")
                time.sleep(wait_time)
            else: 
                pass
    
    #Creating DDBB and geography column
    df_MG["DDBB"] = "Oracle Physics"
    df_MG["geography"] = "Global"
    
    print(df_MG.shape)

    df_MG["UUAA"] = df_MG["UUAA"].astype(str)
    df_MG["Petición Jira"] = df_MG["Petición Jira"].astype(str)
    df_MG["Versión Repo"] = df_MG["Versión Repo"].astype(str)
    df_MG["Petición ARQ"] = df_MG["Petición ARQ"].astype(str)
    df_MG["Fecha cierre"] = df_MG["Fecha cierre"].astype(str)
    df_MG["Comentarios"] = df_MG["Comentarios"].astype(str)
    df_MG["Path"] = df_MG["Path"].astype(str)
    df_MG["DDBB"] = df_MG["DDBB"].astype(str)
    df_MG["geography"] = df_MG["geography"].astype(str)

    df_MG.rename(columns={"Petición Jira": "petition_code", "Versión Repo": "version", "Petición ARQ": "petition_arq", "Fecha cierre": "fecha_out", "Comentarios": "description"}, inplace=True)
    
except gspread.exceptions.APIError as e:
    print(f"APIError: {e}")

except Exception as e:
    print(f"Error: {e}")

Available sheeds: ['KSKR', 'KPDR', 'F30V', 'KAPI', 'H9IY', 'KPAD', 'CHM8', 'XMBR', 'KCOG', 'KONL', 'KNWD', 'KIFV', 'KIFD', 'KPFM', 'KHR1', 'KARC', 'KATT']
(77, 9)


<p style="font-size: 25px; color: #20cff5;">All in one Data Frame</p>

In [7]:
df_g = pd.DataFrame(columns=list(df.columns))
df_g = pd.concat([df_g, df_OG, df_EG, df_MG], ignore_index=True)

columns_to_use = ["fecha_in", "fecha_out", "UUAA", "DQDP_code", "sdatool", "feature", "geography", "DDBB", "Ámbito", "responsable", "validada", "description", "version_date", "version", "petition_arq", "estado", "petition_code", "duration_time", "dev_master"]

dataframes = [df, df_g]
for column in columns_to_use: 
    for i in dataframes:
        i[column] = i[column].astype(str)

df["duration_time"] = pd.to_numeric(df["duration_time"], errors="coerce")
df["dev_master"] = "None"

df_g["duration_time"] = pd.to_numeric(df_g["duration_time"], errors="coerce")
df_g["dev_master"] = "None"

####################################################################################################
dataframes_using = [df, df_g]
for i in dataframes_using:
    for row in i.iterrows(): 
        for key, value in dict(row[1]).items():
            if value == "Nan" or value == None or value == "" or value == "nan" or value == "None" or value == "none" or value == "NONE" or value == np.nan: 
                if key == "fecha_in" or key == "fecha_out" or key == "Fecha incurrida" or key == "UUAA" or key == "DQDP_code" or key == "sdatool" or key == "feature" or key == "geography" or key == "DDBB" or key == "Ámbito" or key == "responsable" or key == "validada" or key == "description" or key == "version_date" or key == "version" or key == "petition_arq" or key == "estado" or key == "petition_code" or key == "dev_master":
                    i.at[row[0], key] = "None"
                if key == "duration_time":
                    i.at[row[0], key] = np.nan
                else: 
                    pass

df_g.drop_duplicates(subset=["petition_code"], keep="first", inplace=True)
df_g = df_g[df_g["petition_code"] != "None"]

#df = pd.concat([df, df_g], ignore_index=True)

In [8]:
df_definitive = pd.DataFrame(columns=columns_to_use)
petitions = set(df_g["petition_code"]).union(set(df["petition_code"]))
print(len(petitions), df_definitive.columns)
petitions

2369 Index(['fecha_in', 'fecha_out', 'UUAA', 'DQDP_code', 'sdatool', 'feature',
       'geography', 'DDBB', 'Ámbito', 'responsable', 'validada', 'description',
       'version_date', 'version', 'petition_arq', 'estado', 'petition_code',
       'duration_time', 'dev_master'],
      dtype='object')


{'OTRSSD-16965',
 'OTRSSD-3861',
 'OTRSSD-4475',
 'SGS-16509',
 'REQ000101524206',
 'OTRSSD-8382',
 'OTRSSD-14251',
 'SGS-23207',
 'OTRSSD-8834',
 'OTRSSD-11317',
 'OTRSSD-17544',
 'SGS-12973',
 'OTRSSD-15082',
 'SGS-21426',
 'OTRSSD-14295',
 'OTRSSD-8339',
 'OTRSSD-14534',
 'SGS-16039',
 'OTRSSD-2079',
 'OTRSSD-16083',
 'OTRSSD-4606',
 'OTRSSD-5195',
 'OTRSSD-14203',
 'OTRSSD-14389',
 'OTRSSD-16540',
 'OTRSSD-15014',
 'OTRSSD-10838',
 'OTRSSD-7776',
 'OTRSSD-17488',
 'OTRSSD-17798',
 'TVPV_20240719134011',
 'SGS-7099',
 'OTRSSD-8776',
 'WO0000101347928',
 'OTRSSD-9232',
 'OTRSSD-16498',
 'SGS-10094',
 'KPSA_20240425095101',
 'OTRSSD-7058',
 'OTRSSD-13189',
 'OTRSSD-16764',
 'OTRSSD-16297',
 'WO0000101350936',
 'OTRSSD-14806',
 'OTRSSD-13506',
 'OTRSSD-16857',
 'OTRSSD-16177',
 'OTRSSD-10697',
 'EBWR_20240604120527',
 'SGS-19984',
 'SGS-17358',
 'REQ000100958660',
 'OTRSSD-6912',
 'OTRSSD-16734',
 'OTRSSD-17487',
 'SGS-18067',
 'OTRSSD-14957',
 'OTRSSD-15696',
 'OTRSSD-18313',
 ' ',
 '

In [9]:
for petition in petitions: 
    dic_aux = {i: [] for i in columns_to_use}

    for column in columns_to_use: 
        df_filtered = df[df["petition_code"] == petition]
        df_g_filtered = df_g[df_g["petition_code"] == petition]
        
        if not df_filtered.empty: 
            df_value = df_filtered[column].values[0]
        else: 
            df_value = "None" if column != "duration_time" else None

        if not df_g_filtered.empty: 
            df_g_value = df_g_filtered[column].values[0]
        else: 
            df_g_value = "None" if column != "duration_time" else None
        

        if df_value != "None" and df_value is not None: 
            dic_aux[column].append(df_value)
        elif df_g_value != "None" and df_g_value is not None: 
            dic_aux[column].append(df_g_value)
        else:
            dic_aux[column].append(df_value) #None o "None"

    df_aux = pd.DataFrame(dic_aux)
    df_definitive = pd.concat([df_definitive, df_aux], ignore_index=True)

C:\Users\matia\AppData\Local\Temp\ipykernel_21128\2068147553.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_definitive = pd.concat([df_definitive, df_aux], ignore_index=True)


In [10]:
dfs = [df, df_g, df_definitive]
df_none_count = {i: 0 for i in list(dfs[2].columns)}
for row in dfs[2].iterrows(): 
    for key, value in dict(row[1]).items():
        if value == "None" or value == None or (isinstance(value, float) and np.isnan(value)):
            df_none_count[key] += 1
        else: 
            pass
print(dfs[0].shape, dfs[1].shape, dfs[2].shape)
df_none_count

(1739, 23) (1138, 24) (2369, 19)


{'fecha_in': 977,
 'fecha_out': 308,
 'UUAA': 2,
 'DQDP_code': 718,
 'sdatool': 803,
 'feature': 778,
 'geography': 205,
 'DDBB': 128,
 'Ámbito': 886,
 'responsable': 1294,
 'validada': 1505,
 'description': 1883,
 'version_date': 2369,
 'version': 1234,
 'petition_arq': 1279,
 'estado': 2369,
 'petition_code': 1,
 'duration_time': 776,
 'dev_master': 2369}

In [11]:
df_definitive

,fecha_in,fecha_out,UUAA,DQDP_code,sdatool,feature,geography,DDBB,Ámbito,responsable,validada,description,version_date,version,petition_arq,estado,petition_code,duration_time,dev_master
0,29/08/2024,29/08/2024,KCNC,DQ-DP-006870,38006,VISTA-988,España,ORACLE Physics,NextGEN,Mario,Si,None,None,None,None,None,OTRSSD-16965,1.0,None
1,None,21/10/2022,KCOG,None,None,None,Global,Oracle Physics,None,None,None,None,None,21,None,None,OTRSSD-3861,NaN,None
2,None,17/11/2022,XMBR,None,None,None,Global,Oracle Physics,None,None,None,None,None,4,DATASD-47790,None,OTRSSD-4475,NaN,None
3,None,24/01/2022,NIOC,None,None,None,Global,Oracle Physics,None,None,None,None,None,5,ARQDAT-33614,None,SGS-16509,NaN,None
4,27/09/2024,30/09/2024,OOKS,DQ-DP-007031,45293,FINCONNEOC-934,España,Oracle R2,Legacy,David,Si,None,None,None,None,None,REQ000101524206,1.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2364,6/11/2024,21/11/2024,XBOB,DQ-DP-007315,44974,WSONEPIV13-1593,Global,Oracle Physics,NextGEN,None,None,3.1.3 v4 master ---> 3.4.3 v4 dev. En 3 tablas...,None,v4 --> v4 dev,WO0000101242243,None,OTRSSD-17992,1.0,None
2365,21/08/2024,21/08/2024,EICF,DQ-DP-006828,45317,SOLPTOVFIS-35,España,ORACLE Physics,NextGEN,Mario,Si,None,None,None,None,None,OTRSSD-16784,1.0,None
2366,25/10/2024,13/11/2024,TEBD,DQ-DP-007245,34546,CIBCALYPS1-451,España,Oracle R2,Legacy,David,None,TAS000019612977. Modificaciones y campos nuevo...,None,None,None,None,REQ000101603119,2.0,None
2367,None,13/9/2021,KSPH,None,None,None,Global,Oracle Physics,None,None,None,Hemos entrado solo en está revisión la versión...,None,6,ARQDAT-31643,None,SGS-13359,NaN,None


---
---

In [12]:
for row in df_definitive.iterrows():
    insert_data_testing(dict(row[1]))

<p style="font-size: 40px; color: grey;">Testing</p>

In [13]:
conn1 = sqlite3.connect("BBVA_testing.db")
cursor1 = conn1.cursor()
cursor1.execute("PRAGMA foreign_keys = ON") #In sqlite3 foreign keys are disabled by default

#######
cursor1.execute("SELECT UUAA FROM UUAA")
all_uuaa = [x[0] for x in cursor1.fetchall()]

#######
conn1.commit()
cursor1.close()
conn1.close()